In [8]:
from pgljupyter import LsystemWidget
from openalea.vmango.preparation.glm_estimation.get_data_for_glm import main as get_data_for_glm
import rpy2.robjects as r
import os

In [1]:
os.chdir('/home/jovyan/src/openalea/vmango/preparation/glm_estimation')
# generate input files located at glm_estimate_input/{cultivar} for prob. tables
get_data_for_glm()

# generate prob. tables at glm_output_proba/{cultivar}
rscript = 'estimate_nb_glm_order_1_proba.r'
r.r(f"source('{rscript}')")

[1] "../src/openalea/vmango/preparation/glm_estimation"
[1] "../../../../../share/glm_output_proba/cogshall/allfactors/"


R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: splines

R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: survival

R[write to console]: Loading required package: TH.data

R[write to console]: Loading required package: MASS

R[write to console]: 
Attaching package: ‘TH.data’


R[write to console]: The following object is masked from ‘package:MASS’:

    geyser


R[write to console]: 
Attaching package: ‘combinat’


R[write to console]: The following object is masked from ‘package:utils’:

    combn




[1] "*** Cycle :  04 ***"
[1] "*** Factors :  Burst_Month , Position_A , Position_Ancestor_A , Nature_Ancestor_F ***"
[1] "determine_vegetative_development"
[1] "########### Estimate Burst"
[1] "Group Burst_Month : 9-10-11 , 12 , 1-2 , 3-4-5"
[1] "########### Estimate Number Children "
[1] "Filter value 1 for factor Burst_Month : 2 -> 456"
[1] "Filter value 2 for factor Burst_Month : 5 -> 451"
[1] "########### Estimate Has_Apical_Child"
[1] "########### Estimate Has_Lateral_Children"
[1] "Filter value 1 for factor Burst_Month : 1 -> 1194"
[1] "Filter value 2 for factor Burst_Month : 5 -> 812"
[1] "########### Estimate Number of Lateral Children "
[1] "Filter value 1 for factor Burst_Month : 2 -> 150"
[1] "########### Estimate Burst Date of Children"
[1] "Group Burst_Month : 9-10 , 11 , 12 , 1 , 2 , 3 , 4 , 5"
[1] "Filter value 1 for factor Burst_Month : 2 -> 456"
[1] "Filter value 2 for factor Burst_Month : 5 -> 451"
[1] "########### Estimate Burst Delta Date of Children with vglm"
[1]

value,[RTYPES.NILSXP]
visible,[RTYPES.LGLSXP]


In [9]:
os.chdir('/home/jovyan/src/openalea/vmango/simulation')
LsystemWidget('mango_simulation.lpy', unit='mm', display_size(800, 600))

LsystemWidget(lsystem=<openalea.lpy.__lpy_kernel__.Lsystem object at 0x7f35573246f8>, scene={'drc': <memory at…